In [31]:
import torch
import numpy as np 
import os
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torchvision.models import squeezenet1_1
import torch.functional as F 
from io import open
from PIL import Image
import cv2
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [32]:
train_path = 'archive\seg_train\seg_train'
pred_path = 'archive\seg_pred\seg_pred'

In [33]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [34]:
#CNN Network
class ConvNet(nn.Module):
    def __init__(self, num_classes=6):
        super(ConvNet,self).__init__() #specifying all layers in neural network

        #output size after convolution filter: (150-3+2(1))/(1) +1

        #Input shape= (256,3,150,150) as 256 is the batch size, 3 is the number of channels(RGB), along with height and width
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #shape = (256,12,150,150) since output channels = 12
        self.bn1 = nn.BatchNorm2d(num_features=12) # adding batch normalization where we are adding the number of features that is equal to number of channels
        #shape remains the same
        self.reul1=nn.ReLU() #brings non linearity 
        #shapes remains the same

        self.pool=nn.MaxPool2d(kernel_size=2) #reduces height and width of our convolution output by a factor of 2
        #shape = (256,12,75,75)

        #add another convolution layer
        self.conv2 = nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #shape = (256,20,75,75) since output channels = 20
        self.reul2=nn.ReLU() #brings non linearity 
        
        #add last layer
        self.conv3 = nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #shape = (256,30,75,75) since output channels = 30
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.reul3=nn.ReLU() #brings non linearity

         #add fully connected layer where we feed the number of input features 
        self.fc=nn.Linear(in_features=32*75*75,out_features=num_classes)

        #feed foward function
    def forward(self,input): #pass in all inputs from above layers
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.reul1(output)

        output=self.pool(output)

        output=self.conv2(output)
        output=self.reul2(output)
        
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.reul3(output)

        #Output will be in Matrix form with shape of (256,32,75,75)

        output=output.view(-1,32*75*75) #in order to feed into full connected layer we reshape into a vector

        output=self.fc(output) #feed into fully connected layer 

        return output

In [35]:
checkpoint = torch.load('best_checkpoint.model')
model  = ConvNet(num_classes=6)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (reul1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (reul2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (reul3): ReLU()
  (fc): Linear(in_features=180000, out_features=6, bias=True)
)

In [36]:
 #Transforming the Size of All Images
#All for all images we add the possibiliy of horizontally flipping our images randomly to add VARIATION to our sets
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(), #changes to pixel range of each color channel from 0-255 to 0-1 and changes data type from numpy to tensors
    #because pyTorch framework takes Tensor as input
    transforms.Normalize([.5,.5,.5], #new pixels will be calculate as (x-mean)/standard deviation where x is the old pixel value
                         [.5,.5,.5]) #changes range from 0-1 to -1-1
    #column represents RGB channel and row represents mean and divisoin, so they are .5 for all channels
])

In [37]:
#prediciton function
def prediction(img_path,transformer):
    image = Image.open(img_path) #read image
    image_tensor = transformer(image).float() #feed to transformer so it converts to tensor
    image_tensor=image_tensor.unsqueeze_(0) #pytorch reads all images as batches so we have to add extra dimension
    if torch.cuda.is_available():
        image_tensor.cuda()
    input = Variable(image_tensor) #convert to variable
    output = model(input) #returns array for each category 
    index = output.data.numpy().argmax() #apply to get category iD
    pred= classes[index] #feed index into classes 
    return pred # return category name

In [38]:
images_path = glob.glob(pred_path+'/*.jpg')

In [39]:
pred_dict = {}
for i in images_path:
     pred_dict[i[i.rfind('/')+1:]] = prediction(i,transformer) #saves image name and prediction

In [40]:
pred_dict

{'archive\\seg_pred\\seg_pred\\10004.jpg': 'street',
 'archive\\seg_pred\\seg_pred\\10005.jpg': 'mountain',
 'archive\\seg_pred\\seg_pred\\10012.jpg': 'buildings',
 'archive\\seg_pred\\seg_pred\\10013.jpg': 'glacier',
 'archive\\seg_pred\\seg_pred\\10017.jpg': 'mountain',
 'archive\\seg_pred\\seg_pred\\10021.jpg': 'forest',
 'archive\\seg_pred\\seg_pred\\1003.jpg': 'sea',
 'archive\\seg_pred\\seg_pred\\10034.jpg': 'glacier',
 'archive\\seg_pred\\seg_pred\\10038.jpg': 'sea',
 'archive\\seg_pred\\seg_pred\\10040.jpg': 'street',
 'archive\\seg_pred\\seg_pred\\10043.jpg': 'sea',
 'archive\\seg_pred\\seg_pred\\10045.jpg': 'street',
 'archive\\seg_pred\\seg_pred\\10047.jpg': 'mountain',
 'archive\\seg_pred\\seg_pred\\10048.jpg': 'buildings',
 'archive\\seg_pred\\seg_pred\\10052.jpg': 'buildings',
 'archive\\seg_pred\\seg_pred\\10054.jpg': 'glacier',
 'archive\\seg_pred\\seg_pred\\10059.jpg': 'forest',
 'archive\\seg_pred\\seg_pred\\10060.jpg': 'forest',
 'archive\\seg_pred\\seg_pred\\10066.j